In [1]:
import os
#from constants import openai_key
from langchain.llms import OpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain, SequentialChain

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

from langchain.callbacks.tracers import ConsoleCallbackHandler

import pandas as pd
import re

import random

import time

import pickle

from tqdm import tqdm



/Users/andrzejrostkowski/python-code/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
OpenAI.api_key = os.environ["OPENAI_API_KEY"]

In [3]:
with open('df_checkpoint_colab.pkl', 'rb') as f:
    df = pickle.load(f)

df.head()


,stars,reviewText,name,categories,Prompty,Wygenerowane2
0,3,"If you decide to eat here, just be aware it is...",Turning Point of North Wales,"Restaurants, Breakfast & Brunch, Food, Juice B...",Generate an internet review for a restaurant w...,I've visited this restaurant chain across mult...
1,5,I've taken a lot of spin classes over the year...,Body Cycle Spinning Studio,"Active Life, Cycling Classes, Trainers, Gyms, ...",Generate an internet review for Body Cycle bas...,I recently discovered Body Cycle and I am blow...
2,3,Family diner. Had the buffet. Eclectic assortm...,Kettle Restaurant,"Restaurants, Breakfast & Brunch",Generate an internet review for the family din...,The family diner next to Clarion Hotel offers ...
3,5,"Wow! Yummy, different, delicious. Our favo...",Zaika,"Halal, Pakistani, Restaurants, Indian",Impress others with your review of a restauran...,"Tucked away, this restaurant surprises with ex..."
4,4,Cute interior and owner (?) gave us tour of up...,Melt,"Sandwiches, Beer, Wine & Spirits, Bars, Food, ...",Generate an internet review for a visited busi...,I recently visited this charming cafe and was ...


In [32]:
start_time = time.time()

range_of_numbers = tqdm(range(9001, 10000))  # Replace with your desired range of numbers
for nr in range_of_numbers:
    max_length = len(re.findall(r'\w+', df['reviewText'][nr]))

    categories=df['categories'][nr]
    exampleReview=df['reviewText'][nr]
    stars=df['stars'][nr]

    hot_model = ChatOpenAI(temperature=0.9)
    model = ChatOpenAI(temperature=0.7)

    researcher_prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You are a researcher. List all the facts you know about a given business based on provided categories and example review. The output should be a short list.",
            ),
            ("human", "Business categories for context: {categories}\n\n\Example review: {exampleReview}"),
        ]
    )

    linguistic_prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                #"You are a linguist specializing in text analysis. For the review text provided, identify and describe the comprehensive linguistic features. Focus on aspects like sentiment, syntax, pragmatics, and lexical choices without suggesting specific words. Aim to explore the depth of the text's linguistic structure in a detailed list. Consider elements like emotional tone, variety and complexity of sentence structures, usage of specific types of words (like jargon or colloquialisms), and rhetorical devices if present."
                "You are a linguist. Analyze the linguistic aspects of the following review text. List all the key linguistic features that could describe this review, without suggesting exact words. The output should be a short list.",
            ),
            ("human", "{exampleReview}"),
        ]
    )

    prompting_prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You are a prompt engineer. Create a prompt for ChatGPT to generate an internet review for a visited business. Use to the provided business information for context. The prompt must match the linguistic instructions provided. Be creative and keep the prompt short, concise and informative. Do not create a review yourself, create a prompt to generate the review instead.",
            ),
            ("human", """Business information: "{researchInstructions}"\n\nLinguistic instructions: "{linguisticInstructions}" """),
        ]
    )

    pre_writing_prompt = PromptTemplate.from_template(
        template = f"Limit your response to {max_length} words. Ensure your review reflects a rating of {stars} stars without mentioning that in the review.",
        template_format="f-string",
    )

    writing_prompt = ChatPromptTemplate.from_messages([
        ("human", "{finalPrompt})"),
        pre_writing_prompt.format(stars=stars,max_length=max_length),

    ])

    researcher_chain = researcher_prompt | hot_model | StrOutputParser()
    linguistic_chain = linguistic_prompt | hot_model | StrOutputParser()

    map_chain = RunnableParallel(researchInstructions=researcher_chain, linguisticInstructions=linguistic_chain)

    prompting_chain = prompting_prompt | model | StrOutputParser()
    writing_chain = {"finalPrompt" : RunnablePassthrough() } | writing_prompt | model | StrOutputParser()

    chain = (map_chain
            | {"finalPrompt" : prompting_chain}
            | RunnablePassthrough.assign(generatedReview=writing_chain))

    result = chain.invoke({"exampleReview": exampleReview, "categories": categories})

    #print(result['finalPrompt'])
    #print('Gen: ' + str(result['generatedReview']))
    df.loc[nr, 'Prompty'] = result['finalPrompt']
    df.loc[nr, 'Wygenerowane2'] = result['generatedReview']


    #printowanie

    if nr % 100 == 0:
        print(f'Processed {nr} record')
        execution_time = time.time() - start_time
        minutes = execution_time // 60
        seconds = execution_time % 60
        print(f"Czas wykonania: {int(minutes)} minut {seconds} sekund")

 10%|█         | 100/999 [57:16<8:33:28, 34.27s/it]

Processed 9100 record
Czas wykonania: 57 minut 16.626972675323486 sekund


 20%|██        | 200/999 [1:54:58<7:59:02, 35.97s/it]

Processed 9200 record
Czas wykonania: 114 minut 58.32630157470703 sekund


 30%|███       | 300/999 [2:52:28<6:37:25, 34.11s/it]

...

Processed 9400 record
Czas wykonania: 230 minut 2.2148799896240234 sekund


 50%|█████     | 500/999 [4:47:41<4:53:53, 35.34s/it]

Processed 9500 record
Czas wykonania: 287 minut 41.768474817276 sekund


 55%|█████▌    | 554/999 [5:19:03<4:16:16, 34.55s/it]


KeyboardInterrupt: 

### Podejscie bardziej optymalne

In [47]:
start_time = time.time()
from langchain.globals import set_debug

set_debug(False)

range_of_numbers = tqdm(range(40000, 40021))  # Replace with your desired range of numbers

hot_model = ChatOpenAI(temperature=0.9)
model = ChatOpenAI(temperature=0.7)

researcher_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a researcher. List all the facts you know about a given business based on an example review. The output should be a short list.",
        ),
        ("human", "Example review: {exampleReview}"),
    ]
)

linguistic_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            #"You are a linguist specializing in text analysis. For the review text provided, identify and describe the comprehensive linguistic features. Your goal is to create instructions that allow the review to be replicated. Focus on aspects like sentiment, syntax, pragmatics, and lexical choices. Consider elements like emotional tone, variety and complexity of sentence structures, usage of specific types of words (like jargon or colloquialisms), and rhetorical devices if present. The output should be a short list."
            "You are a linguist. Analyze the linguistic aspects of the following review text. List all the key linguistic features that could describe this review, including the use of specific words, misspellings and distinct text features that indicate it is a review written by a human. The output should be a short list.",
        ),
        ("human", "{exampleReview}"),
    ]
)

# prompting_prompt = ChatPromptTemplate.from_messages(
#     [
#         (
#             "system",
#             "You are a prompt engineer. Create a prompt for ChatGPT to generate an internet review for a visited business. Use to the provided business information for context. The prompt must match the linguistic instructions provided. Be creative and keep the prompt short, concise and informative. Do not create a review yourself, create a prompt to generate the review instead.",
#         ),
#         ("human", """Business information: "{researchInstructions}"\n\nLinguistic instructions: "{linguisticInstructions}" """),
#     ]
# )

researcher_chain = researcher_prompt | hot_model | StrOutputParser()
linguistic_chain = linguistic_prompt | hot_model | StrOutputParser()

map_chain = RunnableParallel(researchInstructions=researcher_chain, linguisticInstructions=linguistic_chain)

#prompting_chain = prompting_prompt | model | StrOutputParser()

for nr in range_of_numbers:
    max_length = len(re.findall(r'\w+', df['reviewText'][nr]))

    categories=df['categories'][nr]
    exampleReview=df['reviewText'][nr]
    stars=df['stars'][nr]

    pre_writing_prompt = PromptTemplate.from_template(
        template = f"""Write an internet review based on provided business and linguistic instructions. Limit your response to {max_length} words.""",
        template_format="f-string",
    )

    writing_prompt = ChatPromptTemplate.from_messages([
        pre_writing_prompt.format(stars=stars,max_length=max_length),
        ("human", """Business information: "{researchInstructions}"\n\nLinguistic instructions: "{linguisticInstructions}"""),
    ])

    writing_chain = {"researchInstructions" : RunnablePassthrough(), "linguisticInstructions" : RunnablePassthrough()} | writing_prompt | model | StrOutputParser()

    chain = (map_chain
            | {"finalReview" : writing_chain})

    result = chain.invoke({"exampleReview": exampleReview, "categories": categories})

    print(exampleReview)
    print(result['finalReview'])
    #print('Gen: ' + str(result['generatedReview']))
    #df.loc[nr, 'Prompty'] = result['finalReview']
    df.loc[nr, 'Wygenerowane2'] = result['finalReview']

    #printowanie

    if nr % 100 == 0:
        print(f'Processed {nr} record')
        execution_time = time.time() - start_time
        minutes = execution_time // 60
        seconds = execution_time % 60
        print(f"Czas wykonania: {int(minutes)} minut {seconds} sekund")

  5%|▍         | 1/21 [00:03<01:03,  3.15s/it]

It was a wait,but worth it I enjoy every bitethe pancake was ok
The pancakes were worth the wait, but the quality was just average overall.
Processed 40000 record
Czas wykonania: 0 minut 3.167936086654663 sekund


 10%|▉         | 2/21 [00:06<01:05,  3.43s/it]

This place has gone downhill. Extremely slow service, always out of ingredients, always get the wrong order. Probably won't be back.
This place won't be on my list again. Service was slow, ingredients missing, orders wrong - extremely disappointing experience.


 14%|█▍        | 3/21 [00:09<00:59,  3.32s/it]

...

I go here every time I visit family. Best sushi in the area no doubt. It's a small up-and-coming place but perfect atmosphere for date nights. The food speaks for itself, I mean, look at the reviews Sushi Ushi has. The nigiri and rolls aren't pack with too much rice like some places. Every time I get the tako su appetizer. The octopus are generous pieces and always fresh. But today I was surprised with a toro tartar by the sushi chef, soo good! Can't wait to bring my girlfriend for dinner here tonight.
I recently dined at Sushi Ushi, and it truly lives up to its reputation as the best sushi in the area. The intimate setting makes it ideal for date nights. The nigiri and rolls are perfectly balanced, not overloaded with rice. Don't miss the must-try tako su appetizer with generous and fresh octopus. The highlight was the toro tartar surprise from the talented sushi chef. Each visit is a delightful experience, and I can't wait to bring my girlfriend here for dinner tonight. Sushi Ushi ha

 95%|█████████▌| 20/21 [01:25<00:04,  4.75s/it]

Delivery of unit as scheduled ok but pod is damaged noticed mustiness on a few items on delivery day two days later  it rains in unit and collects near back and we have to scramble to get boxes out that we were planning on keeping on the pod and having picked up to be stored for at least two months-- customer service says they will call back they don't we call repeatedly. They are refusing to consider refunding us any charges related to the unit itself . The unit leaked prior to delivery to our address based on what we saw first then leaked more causing further damage . Mr. Mckellar of Pods refuses to consider on behalf of Pods ANY reimbursement for the faulty unit or for us having to relocate our things. No initiative from Pods on how to handle the situation.:(He said we can only file a claim - based on others this has gone nowhere- let's see what happens. BIG inconvenience and cost for a faulty product.Don't use Pods.
I recently used Pods for storage unit delivery, and my experience 

100%|██████████| 21/21 [01:31<00:00,  4.35s/it]

Hmmm.  I hoped to enjoy this place - and above all, the service was excellent during my lunch visit.  The menu is interesting if a little pricey.  I enjoyed the carrot/ginger soup and a fresh green salad.  I also had the crabmeat flat bread, it was riddled with (presumably) crab shell.  Inedibley riddled.  In Melodia's defense, I was the guest of a friend, we were talking up a storm and did not want to address the issue at that time. Given the attentive service, I have no reason to doubt that the matter would have been appropriately remedied had I brought it to their attention.  But really, how off does the kitchen have to be on a given day to make that kind of mistake on one of its signature dishes?  It seems the cook may actually be attempting to sabotage the business.
I recently dined at Melodia for lunch, and hmmm, the service was excellent! I hoped for a great experience, and above all, the staff was attentive and responsive. The menu was interesting, but a little pricey. I enjoye

In [76]:
df['Wygenerowane2'][14913]

'I recently went on a boat tour along the Mississippi River and it was a blast! The live jazz music onboard added a fun vibe, and the lunch buffet was delicious. I also appreciated the educational aspect of the tour, learning about the history of the river. The whole experience was fantastic, but I wish the tour was a bit longer. Overall, a great way to spend a sunny afternoon on the water.'

In [20]:
df.to_pickle('df_checkpoint_vscode_v2.pkl')

In [21]:
import numpy as np
reviews_generated7_full = df.reset_index()
reviews_generated7_full['Wygenerowane2'] = reviews_generated7_full['Wygenerowane2'].replace('', np.nan)
reviews_generated7_full['text_'] = np.where(reviews_generated7_full['Wygenerowane2'].notnull(), reviews_generated7_full['Wygenerowane2'], reviews_generated7_full['reviewText'])
reviews_generated7_full['label'] = np.where(reviews_generated7_full['Wygenerowane2'].notnull(), 'CG', 'OG')
reviews_generated7_full['target'] = np.where(reviews_generated7_full['Wygenerowane2'].notnull(), 1, 0)

print(reviews_generated7_full)
reviews_generated7_full.to_excel('reviews_generated7_full.xlsx', index=True)
reviews_generated7_full.to_csv('reviews_generated7_full.csv', index=True)

       index  stars                                         reviewText  \
0          0      3  If you decide to eat here, just be aware it is...   
1          1      5  I've taken a lot of spin classes over the year...   
2          2      3  Family diner. Had the buffet. Eclectic assortm...   
3          3      5  Wow!  Yummy, different,  delicious.   Our favo...   
4          4      4  Cute interior and owner (?) gave us tour of up...   
...      ...    ...                                                ...   
99995  99995      5  Came in for my 5-6 month prophy and saw Kara -...   
99996  99996      3  Why 3 stars?I love Target, but here is just so...   
99997  99997      4  The Local Taco is the first joint I've found i...   
99998  99998      5  Top notch! He will cater our wedding and  Fran...   
99999  99999      5  Best place to get a vietnamese sandwich and bo...   

                                   name  \
0          Turning Point of North Wales   
1            Body Cycle S

In [24]:
reviews_generated7_cut = reviews_generated7_full[['text_', 'label', 'target']].iloc[30000:40000]
reviews_generated7_cut.columns = ['text', 'label', 'target']
reviews_generated7_cut.to_csv('reviews_generated7_cut.csv', index=True)
reviews_generated7_cut.to_excel('reviews_generated7_cut.xlsx', index=True)
print(reviews_generated7_cut)

                                                    text label  target
30000  I was so disappointed with the hard-fried fish...    CG       1
30001  I recently tried the pho at this spot and it w...    CG       1
30002  So I stumbled upon Cafe Deluxe and boy, was I ...    CG       1
30003  I recently had the pleasure of staying at The ...    CG       1
30004  Our last 3 trips were through AAA and never ha...    CG       1
...                                                  ...   ...     ...
39995  Tony Blanche, owner of the Clam Tavern, has op...    OG       0
39996  a very interesting change of pace for breakfas...    OG       0
39997  This place is just "OK" in my book. I'll go if...    OG       0
39998  Stayed here for a meeting we had at Vanderbilt...    OG       0
39999  I was unexpectedly surprised with how much I l...    OG       0

[10000 rows x 3 columns]
